# Download the images from the tweets and push to s3 for scalable CDN
(Reference: https://github.com/INFO-4602-5602/final-project-tweetsonamap/issues/2)

In [94]:
import pymongo, json, pprint, urllib.request
from html.parser import HTMLParser

Connect to the DB

    client = pymongo.MongoClient('mongodb://epic-analytics.cs.colorado.edu')
    db = client.matthew
    print(db.collection_names())
    tweets = db.tweets
    print(tweets.count(), "tweets")

In [97]:
client = pymongo.MongoClient('localhost')
tweets = client['matthew'].tweets

In [104]:
err_log = open('err_ids.log','w')

###  Image Handlers

In [98]:
class InstagramHandler(HTMLParser):
    """
        Custom function to handle scraping images from Instagram... totally violates TOS, I think.
    """
    def __init__(self, tweet_id):
        self.image_url = None
        self.tweet_id = tweet_id
        self.directory = "instagram"
        super().__init__()

    def handle_starttag(self, tag, attrs):
        if tag=='meta':
            if attrs[0][1]=='og:image':
                self.image_url = attrs[1][1]
                
                try:
                    req = urllib.request.urlopen(self.image_url)
                    with open(self.directory+"/"+self.tweet_id+".jpg",'wb') as pic:
                        pic.write(req.read())
                except:
                    err_log.write("Error, image not found: "+self.tweet_id+"\n")

In [101]:
cursor = tweets.find({'geo' : {"$exists":True}})

In [105]:
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    for url in t['twitter_entities']['urls']:
        print(".",end="")
        
        #Instagram Case
        if url['expanded_url'].startswith("https://www.instagram.com"):            
            parser = InstagramHandler(tweet_id)
            try:
                req = urllib.request.urlopen(url['expanded_url'])
                parser.feed(req.read().decode('utf-8'))
            except:
                err_log.write("Error, page not found: "+tweet_id)

..

-c:8: DeprecationWarning: The value of convert_charrefs will become True in 3.5. You are encouraged to set the value explicitly.


[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/s640x640/e15/14482793_180539412382800_1150376259742597120_n.jpg')]
...[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/s750x750/sh0.08/e35/14474550_300109070358925_5318753938768396288_n.jpg')]
.[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/14449265_1782022908711993_5132982958105296896_n.jpg')]
.[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/e35/14482896_191501847920148_4847263368295219200_n.jpg')]
.[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/e35/14309865_1764163927156341_6997135606522314752_n.jpg')]
..[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-15/e35/14310650_196031957492960_6899400030814208000_n.jpg')]
.[('property', 'og:image'), ('content', 'https://scontent.cdninstagram.com/t51.2885-1

CursorNotFound: Cursor not found, cursor id: 97438851607